In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from preprocess_for_training2 import preprocess
from tensorflow.keras import regularizers

In [2]:
X_train, X_test, y_train, y_test = preprocess()

c:\Users\Bryan\miniconda3\envs\tf_gpu\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Bryan\miniconda3\envs\tf_gpu\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\Users\Bryan\Work\Thaddeus\Data-Analysis\Data Gathering\Staff Analysis Data\ML Model\Train\preprocess_for_training2.py:223: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(vectors)


In [3]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [4]:
# Define the model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.05)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Sigmoid activation for binary classification

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

Epoch 1/50
106/106 [==============================] - 3s 6ms/step - loss: 3.0657 - accuracy: 0.6288 - val_loss: 1.6558 - val_accuracy: 0.7778
Epoch 2/50
106/106 [==============================] - 0s 5ms/step - loss: 1.1210 - accuracy: 0.8064 - val_loss: 0.7403 - val_accuracy: 0.8392
Epoch 3/50
106/106 [==============================] - 0s 4ms/step - loss: 0.6026 - accuracy: 0.8549 - val_loss: 0.4836 - val_accuracy: 0.8818
Epoch 4/50
106/106 [==============================] - 1s 5ms/step - loss: 0.4580 - accuracy: 0.8668 - val_loss: 0.4171 - val_accuracy: 0.8818
Epoch 5/50
106/106 [==============================] - 0s 4ms/step - loss: 0.4083 - accuracy: 0.8733 - val_loss: 0.3882 - val_accuracy: 0.8842
Epoch 6/50
106/106 [==============================] - 0s 4ms/step - loss: 0.3953 - accuracy: 0.8810 - val_loss: 0.3781 - val_accuracy: 0.8936
Epoch 7/50
106/106 [==============================] - 0s 4ms/step - loss: 0.3749 - accuracy: 0.8899 - val_loss: 0.3674 - val_accuracy: 0.8889
Epoch 

In [5]:
# Evaluation
from sklearn.metrics import classification_report, roc_auc_score

# Predict probabilities for validation data
y_pred_proba = model.predict(X_test)

# Convert probabilities to binary predictions
y_pred = (y_pred_proba > 0.5).astype(int)

# Print classification report
print(classification_report(y_test, y_pred))

# Calculate ROC-AUC score
roc_auc = roc_auc_score(y_test, y_pred_proba)
print("ROC-AUC Score:", roc_auc)

              precision    recall  f1-score   support

         0.0       0.82      0.97      0.89       287
         1.0       0.95      0.76      0.84       241

    accuracy                           0.87       528
   macro avg       0.89      0.86      0.87       528
weighted avg       0.88      0.87      0.87       528

ROC-AUC Score: 0.8831089970650744


In [5]:
model.save('mlp_model.h5')  